In [1]:
#exercice 1 

from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum as spark_sum

spark = SparkSession.builder \
    .appName("Traitement de données PySpark") \
    .getOrCreate()

df = spark.read.csv("ventes2.csv", header=True)

df.show()

df = df.withColumn("Chiffre_d_affaires", col("Quantité") * col("Prix_unitaire"))
df.show()

chiffre_d_affaires_total = df.agg(spark_sum("Chiffre_d_affaires").alias("Chiffre_d_affaires_total")).collect()[0]["Chiffre_d_affaires_total"]
print(f"Chiffre d'affaires total : {chiffre_d_affaires_total}")

produit_plus_vendu = df.groupBy("Produit").agg(spark_sum("Quantité").alias("Total_Quantité")) \
    .orderBy(col("Total_Quantité").desc()) \
    .first()

print(f"Produit le plus vendu : {produit_plus_vendu['Produit']} avec {produit_plus_vendu['Total_Quantité']} unités vendues.")

spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/18 11:55:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/18 11:55:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/18 11:55:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Date, Produit, Quantité, Prix_unitaire, 
 Schema: Date, Produit, Quantité, Prix_unitaire, _c4
Expected: _c4 but found: 
CSV file: file:///Users/badrelabbady/Desktop/exo1/ventes2.csv
24/12/18 11:55:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Date, Produit, Quantité, Prix_unitaire, 
 Schema: Date, Produit, Quantité, Prix_unitaire, _c4
Expected: _c4 but found: 
CSV file: file:///Users/badrelabbady/Desktop/exo1/ventes2.csv


+----------+----------+--------+-------------+----+
|      Date|   Produit|Quantité|Prix_unitaire| _c4|
+----------+----------+--------+-------------+----+
|2024-01-01|Ordinateur|       2|          800|NULL|
|2024-01-01|   Clavier|       5|           20|NULL|
|2024-01-02|    Souris|      10|           15|NULL|
|2024-01-02|Ordinateur|       1|          800|NULL|
|2024-01-03|   Clavier|       3|           20|NULL|
+----------+----------+--------+-------------+----+

+----------+----------+--------+-------------+----+------------------+
|      Date|   Produit|Quantité|Prix_unitaire| _c4|Chiffre_d_affaires|
+----------+----------+--------+-------------+----+------------------+
|2024-01-01|Ordinateur|       2|          800|NULL|            1600.0|
|2024-01-01|   Clavier|       5|           20|NULL|             100.0|
|2024-01-02|    Souris|      10|           15|NULL|             150.0|
|2024-01-02|Ordinateur|       1|          800|NULL|             800.0|
|2024-01-03|   Clavier|       3|  

In [2]:
#exercice 2 

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col

spark = SparkSession.builder.appName("Analyse Utilisateurs").getOrCreate()

fichier_json = "utilisateurs.json"
df = spark.read.json(fichier_json)

df_cleaned = df.filter(df.nom.isNotNull() & df.âge.isNotNull() & df.ville.isNotNull())

age_moyen = df_cleaned.select(avg("âge").alias("Âge Moyen")).first()["Âge Moyen"]

utilisateurs_par_ville = df_cleaned.groupBy("ville").count()

plus_jeune = df_cleaned.orderBy("âge").first()

print(f"Âge moyen : {age_moyen:.1f} ans")
print("Nombre d'utilisateurs par ville :")
utilisateurs_par_ville.show()
print(f"Le plus jeune utilisateur est {plus_jeune['nom']} ({plus_jeune['âge']} ans).")

24/12/18 11:55:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Âge moyen : 30.4 ans
Nombre d'utilisateurs par ville :
+---------+-----+
|    ville|count|
+---------+-----+
|Marseille|    1|
|    Paris|    2|
|     Lyon|    2|
+---------+-----+

Le plus jeune utilisateur est Emma (22 ans).


In [21]:
#exercice 3 

from pyspark.sql import SparkSession

# Initialiser SparkSession
spark = SparkSession.builder.appName("Nettoyage de données").getOrCreate()

# Données
data = [("Alice", 25, "Paris", 50000),
        ("Bob", None, "Lyon", 40000),
        ("Claire", 35, "Marseille", 35000),
        ("David", 40, None, 45000),
        ("Emma", 22, "Lyon", None)]
columns = ["Nom", "Age", "Ville", "Revenu"]  # Renommé Âge en Age

# Créer le DataFrame
df = spark.createDataFrame(data, columns)

# Calcul de la moyenne d'âge
age_moyen = df.selectExpr("avg(Age) as age_moyen").collect()[0][0]

# Remplir les valeurs manquantes
df = df.fillna({"Age": age_moyen, "Ville": "Inconnue"})
df = df.na.drop(subset=["Revenu"])

# Afficher le DataFrame nettoyé
df.show()


+------+---+---------+------+
|   Nom|Age|    Ville|Revenu|
+------+---+---------+------+
| Alice| 25|    Paris| 50000|
|   Bob| 30|     Lyon| 40000|
|Claire| 35|Marseille| 35000|
| David| 40| Inconnue| 45000|
+------+---+---------+------+



In [19]:
#exercice 4 

from pyspark.sql.functions import col

data = [("Ordinateur", "Électronique", 800),
        ("Clavier", "Électronique", 20),
        ("Souris", "Électronique", 15),
        ("Table", "Bureau", 150),
        ("Chaise", "Bureau", 80),
        ("Imprimante", "Électronique", 120)]
columns = ["Produit", "Catégorie", "Prix"]

df = spark.createDataFrame(data, columns)

# Trouver les 3 produits les plus chers
produits_plus_chers = df.orderBy(col("Prix").desc()).limit(3)

produits_plus_chers.show()


+----------+------------+----+
|   Produit|   Catégorie|Prix|
+----------+------------+----+
|Ordinateur|Électronique| 800|
|     Table|      Bureau| 150|
|Imprimante|Électronique| 120|
+----------+------------+----+



In [20]:
#Exercice 5

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialiser SparkSession
spark = SparkSession.builder.appName("Analyse des transactions").getOrCreate()

# Données
data = [("Alice", "2024-01-01", 150),
        ("Bob", "2024-01-02", 200),
        ("Alice", "2024-01-03", 100),
        ("Emma", "2024-01-04", 300),
        ("David", "2024-01-05", 250),
        ("Emma", "2024-01-06", 50)]
columns = ["Client", "Date", "Montant"]

# Créer le DataFrame
df = spark.createDataFrame(data, columns)

# Calcul des dépenses totales
depenses_totales = df.groupBy("Client").sum("Montant").orderBy(col("sum(Montant)").desc())

# Client ayant dépensé le plus
meilleur_client = depenses_totales.first()

depenses_totales.show()
print(f"Client ayant dépensé le plus : {meilleur_client['Client']} ({meilleur_client['sum(Montant)']} €)")

+------+------------+
|Client|sum(Montant)|
+------+------------+
|  Emma|         350|
| Alice|         250|
| David|         250|
|   Bob|         200|
+------+------------+

Client ayant dépensé le plus : Emma (350 €)
